# Excel Converersion of Dates from US to Euro Format

In [2]:
import pandas as pd
import os
import re
import openpyxl
from openpyxl.styles import Font, Fill, Color
from openpyxl.styles import colors
from openpyxl.styles import numbers
import numpy as np
np.set_printoptions(suppress=True)

from pathlib import Path
from datetime import datetime

In [3]:
path = 'C:/Users/bgeorgiev/Python Scripts - ipynb/ABS Suite Help/SQL Compare'
#path = Path.cwd()
#convertedpath = path+'/Converted'
os.chdir(path)

### Compare DataFrames Function

In [4]:
def excel_diff(df_OLD, df_NEW, index_col):

    #df_OLD = pd.read_excel(path_OLD, index_col=index_col).fillna(0)
    #df_NEW = pd.read_excel(path_NEW, index_col=index_col).fillna(0)

    # Perform Diff
    dfDiff = df_NEW.copy()
    droppedRows = []
    newRows = []
    diffRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    
    for row in dfDiff.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]

                value_NEW = df_NEW.loc[row,col]
                if value_OLD==value_NEW:
                    dfDiff.loc[row,col] = df_NEW.loc[row,col]
                else:
                    dfDiff.loc[row,col] = ('{}-->{}').format(value_OLD,value_NEW)
                    diffRows.append(row)
        else:
            newRows.append(row)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            dfDiff = dfDiff.append(df_OLD.loc[row,:])

    dfDiff = dfDiff.sort_index().fillna('')
    #print(dfDiff)
    print('\nNew Rows:     {}'.format(newRows))
    print('Dropped Rows: {}'.format(droppedRows))

    # Save output and format
    fname = '{} vs {}.xlsx'.format(path_OLD,path_NEW)
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')

    dfDiff.to_excel(writer, sheet_name='DIFF', index=True)
    df_NEW.to_excel(writer, sheet_name=path_NEW, index=True)
    df_OLD.to_excel(writer, sheet_name=path_OLD, index=True)

    diffRows = list(set(diffRows+newRows+droppedRows))
    df_Changes = dfDiff.loc[diffRows,:]
    print('\nChanges:\n{}'.format(df_Changes))

    # get xlsxwriter objects
    workbook  = writer.book
    worksheet = writer.sheets['DIFF']
    worksheet.hide_gridlines(2)
    worksheet.set_default_row(15)

    # define formats
    #date_fmt = workbook.add_format({'align': 'center', 'num_format': 'yyyy-mm-dd'})
    center_fmt = workbook.add_format({'align': 'center'})
    number_fmt = workbook.add_format({'align': 'center', 'num_format': '#,##0.00'})
    cur_fmt = workbook.add_format({'align': 'center', 'num_format': '$#,##0.00'})
    perc_fmt = workbook.add_format({'align': 'center', 'num_format': '0%'})
    
    #grey_fmt = workbook.add_format({'font_color': '#E0E0E0', 'diag_type': 2,'diag_color': 'red'})
    grey_fmt = workbook.add_format({'font_color': '#E0E0E0'})
    #drop_fmt = workbook.add_format({'font_color': 'red', 'bold':True})   
    drop_fmt = workbook.add_format({'font_color': 'white', 'bg_color': 'red'})
    #new_fmt = workbook.add_format({'font_color': '#32CD32','bold':True})
    new_fmt = workbook.add_format({'font_color': 'white','bg_color': 'green'})
    highlight_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})

    # set format over range
    ## highlight changed cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value':'-->',
                                            'format': highlight_fmt})

#     ## highlight unchanged cells
#     worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
#                                             'criteria': 'not containing',
#                                             'value':'-->',
#                                             'format': grey_fmt})
    # highlight new/changed rows
    #for row in range(dfDiff.shape[0]):
    for i,row in enumerate(dfDiff.index):
        #print(row, newRows)
        if row in newRows:
            print('new row', row)
            worksheet.set_row(i+1, 15, new_fmt)
        if row in droppedRows:
            print('dropped rows', row)
            worksheet.set_row(i+1, 15, drop_fmt)

    # save
    writer.save()
    print('\nDone.\n')

#### Set DB config

In [5]:
import pyodbc
server = '10.25.31.117'
databaseWeb = 'ABSWeb_DaimlerUS'
databaseNet = 'ABSDaimler_BUS'
username = 'sqluser'
password = 'mis'
con = 'Yes'
#Connection String
connectionWeb = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+databaseWeb+';UID='+username+';PWD='+ password)
connectionNet = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+databaseNet+';UID='+username+';PWD='+ password)

# cursor = connection.cursor()
# cursor.execute("SELECT * from DEAL_D;")
# row = cursor.fetchone()
# while row:
#     print(row[0])
#     row = cursor.fetchone()

#### Set SQL Query 

In [6]:
_SQL = '''SELECT TRUST_ID, PCD_ACT_D, count(*) as Count, sum(S_invPrinBal) as S_InvPrinBal,  sum(S_invbegPrinBal) as S_InvBegPrinBal
from TRUCK_CS_MASTER
group by TRUST_ID, PCD_ACT_D
order by TRUST_ID, PCD_ACT_D
'''

indexcols = ['TRUST_ID','PCD_ACT_D']

_SQLWeb = _SQL
#_SQLWeb = 'Use [' + databaseWeb + '] '+ _SQL
_SQLNet = _SQL

#print(_SQLWeb, _SQLNet)

#### Run SQL Queries

In [7]:
dfWeb = pd.read_sql_query(_SQLWeb, connectionWeb)
dfNet = pd.read_sql(_SQLNet, connectionNet)
connectionWeb.close()
connectionNet.close()

In [8]:
dfWeb = dfWeb.sort_values(by=indexcols, ascending=True, na_position='first')
dfNet = dfNet.sort_values(by=indexcols, ascending=True, na_position='first')

#### Compare Excels

In [9]:
#sheetnames for the queries to be saved to
path_OLD='Query_Net'
path_NEW = 'Query_Web'
excel_diff(dfNet, dfWeb,indexcols)


New Rows:     []
Dropped Rows: []

Changes:
     TRUST_ID  PCD_ACT_D  Count                S_InvPrinBal  S_InvBegPrinBal
2  DTRT20181  2018-05-31   9425  990663796.91-->990663796.9     1.028435e+09

Done.

